## Production tpr files generate
- some benchmarking needed first
 
 Protocol files are copied from https://github.com/intbio/gmx_protocols/tree/master/amber into
Protocols folder.
If others are needed modify then in the folder.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from myimports import *
display(Markdown(descr))



_ColormakerRegistry()

Folder set to   :  h2a-h2b_tm_30DNA
Project name    :  h2a-h2b_tm_30DNA
SSH host set to : lomo2
GMX cmd set with: module load gromacs/2020.1_nompi
Force field     : amber14sb_parmbsc1_cufix
Force field URL : https://api.github.com/repos/intbio/gromacs_ff/contents/amber14sb_parmbsc1_cufix.ff
Run profile     : lomo2-gromacs2020.3-mpi-gpu_volta2



# MD simulations of H2A-H2B with 30 bp of DNA, tails truncated
- AMBER14SB force field
- 150 mM NaCl
- box 2nm


In [3]:
# %%bash
#rm -rf MDProtocols

In [4]:
#Set protocols URL
# prot_url="https://api.github.com/repos/intbio/gmx_protocols/contents/amber_gmx2020"

In [5]:
# get_files_from_git(prot_url,'MDProtocols/')

## 7_Production run 10 000 ns - prepare files

In [6]:
%%bash -s "$set_gmx_cmd" "$gen_parallel_tpr"
$1
#--out out --err err
mkdir -p GMX_run
gmx grompp -maxwarn 2 -f MDProtocols/7_prod.mdp -c GMX_run/6_equil.gro -p GMX_system/topol.top \
-o GMX_run/7_prod.tpr -v  -r GMX_run/6_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_7_prod.mdp -t GMX_run/6_equil.cpt

#It is useful to generate some files with different velocities for parallel runs
if [ "$2" = True ]
then
for i in `seq 0 15`; do
gmx grompp -maxwarn 2 -f MDProtocols/7_prod.mdp -c GMX_run/6_equil.gro -p GMX_system/topol.top \
-o GMX_run/7_prod_$i.tpr -v  -r GMX_run/6_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_7_prod.mdp -t GMX_run/6_equil.cpt
done
for i in `seq 0 15`; do
mkdir -p GMX_run/$i
cp GMX_run/7_prod_$i.tpr GMX_run/$i/7_prod.tpr
done
fi

processing topology...
turning H bonds into constraints...
turning H bonds into constraints...
turning H bonds into constraints...
turning H bonds into constraints...
turning H bonds into constraints...
turning H bonds into constraints...
turning H bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K
Calculated rlist for 1x1 atom pair-list as 0.837 nm, buffer size 0.037 nm
Set rlist, assuming 4x4 atom pair-list, to 0.801 nm, buffer size 0.001 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 96x96x96, spacing 0.110 0.110 0.110


(Un)Loading module for modern CPU
                      :-) GROMACS - gmx grompp, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

## Transfer files to supercomputer

## Launch,relaunch make a chain of tasks
### NOTE: for relaunch, monitor and data pullback - better to use status.ipynb

Lomonosov-2 queues https://users.parallel.ru/wiki/pages/22-config
compute partition allows max 7 days!.
Then we will resume.
https://hpc.nih.gov/docs/job_dependencies.html

In [7]:
#Relaunch
#!ssh $ssh_host "module load slurm gromacs/2018-gcc; cd ~/_scratch/$folder/GMX_run; sbatch -p compute -t 7-00:00:00 -N 4 -J $folder lomo2_run_cont.sh 7_prod"

## Monitor

Note: It is suggested that for monitoring and relaunch the user makes use of status.ipynb
Parts of conde below are deprecated.

### Pull back data for analysis

### Quick analysis 

Quick view

## QC analysis
### Let's look at energy, temperature pressure